In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from yfQuery import datareader

In [2]:
class DataLoader:
    '''
        Class for loading past stocks prices
        symbol: symbol can be either a single symbol or 
                a list of symbol
        start:  start date
        end:    end date
    '''
    def __init__(self, symbol, start, end):
        # Preload
        self.data = datareader(symbol, start, end)
    
    def get(self, start, end):
        # Return a period of the data
        return self.data.iloc[start: end]

In [3]:
d = DataLoader('AAPL', '2018-01-01', '2019-12-31')

In [4]:
prices = d.data

In [39]:
# Comparing High price to Open price and Close price to create psudo data
high_to_open = prices['High'] / prices['Open'] - 1
high_to_close = prices['High'] / prices['Close'] - 1
swap_high = high_to_close > high_to_open
swap_high.name = 'Swap'
df = pd.concat([prices.loc[:, ['Open', 'High', 'Low', 'Close']], swap_high], axis=1)
df

,Open,High,Low,Close,Swap
Date,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,False
2018-01-03,43.132500,43.637501,42.990002,43.057499,True
2018-01-04,43.134998,43.367500,43.020000,43.257500,False
2018-01-05,43.360001,43.842499,43.262501,43.750000,False
2018-01-08,43.587502,43.902500,43.482498,43.587502,False
...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,True
2019-12-26,71.205002,72.495003,71.175003,72.477501,False
2019-12-27,72.779999,73.492500,72.029999,72.449997,True


In [5]:
# Total normal trading hours: 6.5
# minute, five_minute, fifteen_minute, half_hour,  and one_hour
total_minutes = int(6.5 * 60)
total_5_minutes = int(6.5 * (60 / 5))
total_15_minutes = int(6.5 * (60 / 15))
total_30_minutes = int(6.5 * (60 / 30))
total_hours = 7
print(total_minutes, total_5_minutes, total_15_minutes, total_30_minutes)

390 78 26 13
